<a href="https://colab.research.google.com/github/danwaters/lfi/blob/main/Dan_Waters_CSCE_5218_Final_Project_(Orchestration_Example).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this notebook is to demonstrate an end-to-end prediction involving both the image classifier and the appropriate text generator. 

In [1]:
!git clone https://github.com/danwaters/lfi.git

fatal: destination path 'lfi' already exists and is not an empty directory.


In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import models
from keras import preprocessing
from keras import utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

In [3]:
IMG_HEIGHT = 180
IMG_WIDTH = 180

In [4]:
classifier_model = keras.models.load_model("lfi/saved_models/classifier", compile=False)

test_url = "https://www.rollingstone.com/wp-content/uploads/2018/06/rs-240736-GettyImages-91150536.jpg"
test_path = tf.keras.utils.get_file('dylan', origin=test_url)
class_names = ['beatles', 'bob-dylan', 'michael-jackson', 'zappa']
img = keras.preprocessing.image.load_img(
    test_path, target_size=(IMG_HEIGHT, IMG_WIDTH)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = classifier_model.predict(img_array)
score = tf.nn.softmax(predictions[0])
class_name = class_names[np.argmax(score)]

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_name, 100 * np.max(score))
)

# call the appropriate language generation model
nl_model = keras.models.load_model(f'lfi/saved_models/{artist}_lyrics', compile=True)

This image most likely belongs to bob-dylan with a 100.00 percent confidence.


In [5]:
def generate_text(model, max_sequence_length, seed_text="she", next_words=100, sequence_word_length=6):
  """ This method generates next_words words based on the seed text by
  repeatedly feeding the last sequence_length words into the LSTM to make the
  prediction. It keeps track of every word generated and prints the result."""
  t = Tokenizer()
  text = open('lfi/train/bob-dylan.txt', 'rb').read().decode(encoding='utf-8')
  sentences = text.lower().replace("\r\n", "\n").split("\n")
  # fit the tokenizer
  t.fit_on_texts(sentences)

  # Seed and run predictions
  total_text = seed_text

  for i in range(next_words):
    token_list = t.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], 
                               maxlen=max_sequence_length - 1,
                               padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in t.word_index.items():
      if index == predicted:
        output_word = word
        break
    total_text += " " + output_word
    seed_text = seed_text + " " + output_word
 
    # if seed_text is n words or more, drop the first word in the sequence.
    seed_words = seed_text.split(' ')
    if len(seed_words) >= sequence_word_length:
      seed_text = ' '.join(seed_words[1:])

  out = ""
  for i, w in enumerate(total_text.split(' ')):
    out = out + " " + w
    if i % sequence_word_length == 0 and i > 0: # insert line breaks every 5 words
      out += "\r\n"
  
  return out

In [7]:
out = generate_text(nl_model, 32)
print(out)

 she by heart by you down he
 you what was complete my the
 table back want the trust did
 bleachers they oh mistreat i we
 mixed it’s the cravin’ proved it
 that would time the quit leaves
 it’s back behind hangin’ near and
 this she out many fade know
 ain’t sold know the explain nails
 needs on oh  i’ll i
 wiggle plan walkin' anymore the need
 spark see disillusioned they you just
 you dance me clarkesdale they you
 a terrible narrow ’bove in the
 enough   longin’ i if
 want worked is broken and i'm
 off the happened my
